In [ ]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import clear_output

# Read the CSV file
df = pd.read_csv('route_data.csv', names=['Day', 'Truck', 'X', 'Y', 'Heuristic'])

prev_day_state = "All"
prev_truck_state = "All"

def return_visibility(curr_day, curr_truck, day, truck):
    if curr_day == "All" and curr_truck == "All":
        return True
    elif curr_day == "All":
        return truck == curr_truck
    elif curr_truck == "All":
        return day == curr_day
    else:
        return day == curr_day and truck == curr_truck

def create_trace_plot(curr_day, curr_truck):
    fig = make_subplots(rows=1, cols=2, subplot_titles=["Greedy Heuristic", "Lin-Kernighan Heuristic"])
    days = None
    trucks = None
    if curr_day == "All":
        days = df['Day'].unique()
    else:
        days = [curr_day]

    if curr_truck == "All":
        trucks = df['Truck'].unique()
    else:
        trucks = [curr_truck]
    # Generate traces for each heuristic
    for day in df['Day'].unique():
        for truck in df['Truck'].unique():
            for heuristic in ['Greedy', 'Lin-Kernighan']:
                sub_df = df[(df['Day'] == day) & (df['Truck'] == truck) & (df['Heuristic'] == heuristic)]
                trace = go.Scatter(
                    x=sub_df['X'], y=sub_df['Y'],
                    mode='lines+markers',
                    name=f'Day {day}, Truck {truck}, {heuristic}',
                    legendgroup=f"{heuristic}{day}{truck}",
                    visible = return_visibility(curr_day, curr_truck, day, truck)
                )
                col = 1 if heuristic == 'Greedy' else 2
                fig.add_trace(trace, row=1, col=col)

    fig.show()

# Create dropdown menus for days and trucks
days = ['All'] + sorted(df['Day'].unique().tolist())
trucks = ['All'] + sorted(df['Truck'].unique().tolist())

# Create the first dropdown
dropdown_for_day = widgets.Dropdown(
    options=days,
    description='Day:',
    disabled=False,
)

# Create the second dropdown
dropdown_for_truck = widgets.Dropdown(
    options=trucks,
    description='Truck:',
    disabled=False,
)

# Define the callback functions
def on_change_dropdown_for_day(change):
    global prev_truck_state
    global prev_day_state
    # Code for on click day dropdown change
    if change['type'] == 'change' and change['name'] == 'value':
        create_trace_plot(change['new'], prev_truck_state)
        prev_day_state = change['new']

def on_change_dropdown_for_truck(change):
    global prev_day_state
    global prev_truck_state
    # Code for on click truck dropdown change
    if change['type'] == 'change' and change['name'] == 'value':
        create_trace_plot(prev_day_state,change['new'])
        prev_truck_state = change['new']

# Attach the callback functions to the dropdowns
dropdown_for_day.observe(on_change_dropdown_for_day, names='value')
dropdown_for_truck.observe(on_change_dropdown_for_truck, names='value')

# Display the dropdowns
display(dropdown_for_day, dropdown_for_truck)
create_trace_plot(prev_day_state, prev_truck_state)